In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

options = Options()
options.add_argument("--headless")   
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

youm7_url = "https://www.youm7.com"
urgent_new = "https://www.youm7.com/Section/%D8%A3%D8%AE%D8%A8%D8%A7%D8%B1-%D8%B9%D8%A7%D8%AC%D9%84%D8%A9/65/1"

driver.get(urgent_new)
time.sleep(5)  

for _ in range(3):     
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

soup = BeautifulSoup(driver.page_source, "lxml")

cards = soup.find_all("div", class_="col-xs-12 bigOneSec")

news_list = []

for idx, card in enumerate(cards, start=1):
    try:
        title_tag = card.find("h3").find("a")
        title = title_tag.get_text(strip=True)
        link = title_tag.get("href")
        if not link.startswith("http"):
            link = youm7_url + link

        img_tag = card.find("img")
        image_url = img_tag.get("src") if img_tag else ""

        date_tag = card.find("span", class_="newsDate2")
        pub_date = date_tag.get_text(strip=True) if date_tag else ""

        desc_tag = card.find("p")
        summary = desc_tag.get_text(strip=True) if desc_tag else ""

        driver.get(link)
        time.sleep(2)
        article_soup = BeautifulSoup(driver.page_source, "lxml")

        paragraphs = article_soup.select("div.newsStory p")
        full_text = " ".join([p.get_text(strip=True) for p in paragraphs])

        if not full_text:
            paragraphs = article_soup.find_all("p")
            full_text = " ".join([p.get_text(strip=True) for p in paragraphs])

        news_list.append({
            "Title": title,
            "Link": link,
            "Date": pub_date,
            "Summary": summary,
            "Full_Article": full_text,
            "Image": image_url
        })

        print(f"Scraped ({idx}/{len(cards)}): {title}")

    except Exception as e:
        print(f"Error scraping card {idx}: {e}")

df = pd.DataFrame(news_list)
df.to_csv("youm7_breaking_news.csv", index=False, encoding="utf-8-sig")

print(f"DONE! Scraped {len(df)} news articles")
print("Saved to youm7_breaking_news.csv")

driver.quit()
